# PyKale example: Domain Adaptation on Digits with Lightning

This example is constructed by refactoring the [ADA: (Yet) Another Domain Adaptation library](https://github.com/criteo-research/pytorch-ada), with many domain adaptation algorithms included.

It has been put together to run interactively on online hosting platofrms including [Google Colab](https://colab.research.google.com) or [myBinder](https://mybinder.org), but can also be downloaded and run locally. Follow the [PyKale](https://github.com/pykale/pykale) installation instructions for this.

*ToDo: Description of what this example actually does.*

# Setup

The first few blocks of code are necessary to set up the notebook execution environment and import the required modules, including PyKale.

This checks if the notebook is running on Google Colab and installs required packages.

In [1]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip install pykale[extras] 

    !git clone -b digits-notebook https://github.com/pykale/pykale.git
    %cd pykale/examples/digits_dann_lightn
else:
    print('Not running on CoLab')

Not running on CoLab


This imports required modules.

In [2]:
import logging
import os

from config import get_cfg_defaults
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.utils.data import SequentialSampler
import torchvision

from model import get_model

from kale.loaddata.digits_access import DigitDataset
from kale.loaddata.multi_domain import MultiDomainDatasets
from kale.utils.csv_logger import setup_logger
from kale.utils.seed import set_seed

## Configuration

In this example we provide a [default configuration for domain adaptation problems](https://github.com/pykale/pykale/blob/main/examples/digits_dann_lightn/config.py) which  which is tailored using a [`.yaml` file for the specific application in this example](https://github.com/pykale/pykale/blob/main/examples/digits_dann_lightn/configs/TUTORIAL.yaml).

If GPUs are to be used at runtime, this is specified using a seperate variable. If you are running this example on Google Colab, or on a machine with GPU support, you might [set this](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html#trainer-class-api) to make use of GPU accleration.

The configuration is summarised below the following cell.

In [3]:
cfg_path = "./configs/TUTORIAL.yaml" # Path to `.yaml` config file
gpus = None # GPU settings

cfg = get_cfg_defaults()
cfg.merge_from_file(cfg_path)
cfg.freeze()
print(cfg)

DAN:
  METHOD: CDAN
  RANDOM_DIM: 1024
  USERANDOM: False
DATASET:
  DIMENSION: 784
  NAME: digits
  NUM_CLASSES: 10
  NUM_REPEAT: 1
  ROOT: ../data
  SIZE_TYPE: source
  SOURCE: svhn
  TARGET: usps
  VAL_SPLIT_RATIO: 0.5
  WEIGHT_TYPE: natural
OUTPUT:
  DIR: ./outputs\digits_mnist2usps
  PB_FRESH: None
  ROOT: ./outputs
  VERBOSE: False
SOLVER:
  AD_LAMBDA: True
  AD_LR: True
  BASE_LR: 0.001
  INIT_LAMBDA: 1
  MAX_EPOCHS: 1
  MIN_EPOCHS: 0
  MOMENTUM: 0.9
  NESTEROV: True
  SEED: 2020
  TEST_BATCH_SIZE: 200
  TRAIN_BATCH_SIZE: 150
  TYPE: SGD
  WEIGHT_DECAY: 0.0005


# ---- setup output ----

In [4]:
os.makedirs(cfg.OUTPUT.DIR, exist_ok=True)
format_str = "@%(asctime)s %(name)s [%(levelname)s] - (%(message)s)"
logging.basicConfig(format=format_str)

# ---- setup dataset ----

In [5]:
source, target, num_channels = DigitDataset.get_source_target(
    DigitDataset(cfg.DATASET.SOURCE.upper()), DigitDataset(cfg.DATASET.TARGET.upper()), cfg.DATASET.ROOT
)

dataset = MultiDomainDatasets(
    source,
    target,
    config_weight_type=cfg.DATASET.WEIGHT_TYPE,
    config_size_type=cfg.DATASET.SIZE_TYPE,
    val_split_ratio=cfg.DATASET.VAL_SPLIT_RATIO,
)

# Repeat multiple times to get std

In [6]:
for i in range(0, cfg.DATASET.NUM_REPEAT):
    seed = cfg.SOLVER.SEED + i * 10
    # seed_everything in pytorch_lightning did not set torch.backends.cudnn
    set_seed(seed)
    print(f"==> Building model for seed {seed} ......")
    # ---- setup model and logger ----
    model, train_params = get_model(cfg, dataset, num_channels)
    logger, results, checkpoint_callback, test_csv_file = setup_logger(
        train_params, cfg.OUTPUT.DIR, cfg.DAN.METHOD, seed
    )

    if gpus is None:
        trainer = pl.Trainer(
            progress_bar_refresh_rate=cfg.OUTPUT.PB_FRESH,  # in steps
            min_epochs=cfg.SOLVER.MIN_EPOCHS,
            max_epochs=cfg.SOLVER.MAX_EPOCHS,
            callbacks=[checkpoint_callback],
            logger=False,
        )
    else:
        trainer = pl.Trainer(
            progress_bar_refresh_rate=cfg.OUTPUT.PB_FRESH,  # in steps
            min_epochs=cfg.SOLVER.MIN_EPOCHS,
            max_epochs=cfg.SOLVER.MAX_EPOCHS,
            callbacks=[checkpoint_callback],
            logger=False,
            gpus=gpus,
        )

    trainer.fit(model)
    results.update(
        is_validation=True, method_name=cfg.DAN.METHOD, seed=seed, metric_values=trainer.callback_metrics,
    )
    # test scores
    trainer.test()
    results.update(
        is_validation=False, method_name=cfg.DAN.METHOD, seed=seed, metric_values=trainer.callback_metrics,
    )
    results.to_csv(test_csv_file)
    results.print_scores(cfg.DAN.METHOD)

==> Building model for seed 2020 ......
Using downloaded and verified file: ../data\train_32x32.mat
Using downloaded and verified file: ../data\test_32x32.mat


C:\Users\bobturner\.conda\envs\pykale\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: Checkpoint directory ./outputs\digits_mnist2usps\checkpoints\c8ffda20f933bfb5ff885ff343472ef3\CDAN\seed_2020 exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type                | Params
----------------------------------------------------------
0 | feat              | SmallCNNFeature     | 312 K 
1 | classifier        | ClassNetSmallImage  | 24.4 K
2 | domain_classifier | DomainNetSmallImage | 128 K 
----------------------------------------------------------
466 K     Trainable params
0         Non-trainable params
466 K     Total params
1.864     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

C:\Users\bobturner\.conda\envs\pykale\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


MisconfigurationException: `.test(ckpt_path="best")` is set but `ModelCheckpoint` is not configured to save the best model.